In [2]:
from io import open
# 用于字符规范化
import unicodedata
# 用于正则表达式
import re
# 用于随机生成数据
import random
# 用于构建网络结构和函数的torch工具包
import torch
import torch.nn as nn
import torch.nn.functional as F
# torch中预定义的优化方法工具包
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### 对持久化文件中的数据进行处理，以满足模型训练的要求

In [4]:
print(device)

cpu


In [5]:
# 将指定语言中的词汇映射成数值
# 起始标志
SOS_token = 0
# 结束标志
EOS_token = 1

class Lang:
    def __init__(self, name):
        """初始化函数中参数name代表传入某种语言的名字"""
        # 将name传入类中
        self.name = name
        # 初始化词汇对应自然数值的字典
        self.word2index = {}
        # 初始化词汇对应自然数值的字典,其中0,1对应的SOS和EOS已经在里面了
        self.index2word = {0:"SOS", 1:"EOS"}
        # 初始化词汇对应的自然数索引，这里从2开始，因为0,1已经被开始和结束标志占用了
        self.n_words = 2


    def addSentence(self, sentence):
        """添加句子函数，即将句子转化为对应的数值序列，输入参数sentence是一条句子"""
        # 根据一般国家的语言特性(我们这里研究的语言都是以空格分隔单词)
        # 对句子进行分割，得到对应的词汇列表
        for word in sentence.split(' '):
            # 然后调用addWord进行处理
            self.addWord(word)

    def addWord(self, word):
        """添加词汇函数，即将词汇转化为对应的数值，输入参数word是一个单词"""
        # 首先判断word是否已经在self.word2index字典的key中
        if word not in self.word2index:
            # 如果不在，则将这个词加入其中，并为它对应一个数值，即self.n_words
            self.word2index[word] = self.n_words
            # 同时也将它的反转形式加入到self.index2word中
            self.index2word[self.n_words] = word
            # self.n_words一旦被占用之后逐次加1,变成新的self.n_words
            self.n_words += 1

In [6]:
# 实例化参数
name = "eng"
# 输入参数
sentence = "Hello I am Jay"

# 调用
eng1  = Lang(name)
eng1.addSentence(sentence)
print("word2index:", eng1.word2index)
print("index2word:", eng1.index2word)
print("n_words:", eng1.n_words)

word2index: {'Hello': 2, 'I': 3, 'am': 4, 'Jay': 5}
index2word: {0: 'SOS', 1: 'EOS', 2: 'Hello', 3: 'I', 4: 'am', 5: 'Jay'}
n_words: 6


In [7]:
# 字符规范化
# 将unicode转为Ascii,我们可以认为是去掉一些语言中的重音标记
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

def normalizeString(s):
    """字符串规范化函数，参数s代表传入的字符串"""
    # 使字符变为小写工去除两侧空白符，z再使用unicodeToAsccii去掉重音标记
    s = unicodeToAscii(s.lower().strip())
    # 在.!?前加一个空格
    s = re.sub(r"([.!?])", r" \1", s)
    # 使用正则表达式将字符串中不是大小写字母和正常标点的都替换成空格
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

# 输入参数
s = "Are you kidding me?"

# 调用
nsr = normalizeString(s)
print(nsr)

are you kidding me ?


In [8]:
# 将持久化文件中的数据加载到内存，并实例化类Lang
data_path = './data/eng-fra.txt'

def readLangs(lang1, lang2):
    """读取语言函数，参数lang1是源语言的名字，参数lang2是目标语言的名字， 返回对应的class Lang对象，以及语言对列表"""
    # 从文件中读取语言对并以/n划分存到列表lines中
    lines = open(data_path, encoding='utf-8').\
        read().strip().split('\n')
    # 对lines列表中的句子进行标准化处理，并以\t进行再次划分，形成子列表，也就是语言对
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    # 然后分别将语言名字传入Lang类中，获得对应的语言对象，返回结果
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)
    return  input_lang, output_lang, pairs

In [9]:
# 输入参数
lang1 = "eng"
lang2 = "fra"

# 调用
input_lang, output_lang, pairs = readLangs(lang1, lang2)
print("input_lang:", input_lang)
print("output_lang:", output_lang)
print("parirs中的前五个:", pairs[:5])

input_lang: <__main__.Lang object at 0x7fb3d16ca2e0>
output_lang: <__main__.Lang object at 0x7fb3d5332070>
parirs中的前五个: [['go .', 'va !'], ['run !', 'cours !'], ['run !', 'courez !'], ['wow !', 'ca alors !'], ['fire !', 'au feu !']]


In [11]:
# 过滤出我们需要的语言对
# 设置组成句子中单词或标点的最多个数
MAX_LENGTH = 10

# 选择带有指定前缀的语言特征数据作为训练数据
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    """语言对过滤函数，参数p代表输入的语言对，如['she is afraid.', 'elle malade.']"""
    # p[0]代表英语句子，对它进行划分，它的长度应小于最大长度MAX_LENGTH并且要以指定的前缀开头
    # p[1]代表法文句子，对它进行划分，它的长度应小于最大长度MAX_LENGTH
    return len(p[0].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes) and \
        len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    """对多个语言对列表进行过滤，参数pairs代表语言对组成的列表，简称语言对列表"""
    # 函数中直接遍历列表中的每个语言对并调用filterPair即可
    return [pair for pair in pairs if filterPair(pair)]

# 输入参数paris使用readLangs函数的输出结果pairs
# 调用
fpairs = filterPairs(pairs)
print("过滤后的pairs前五个:", fpairs[:5])

过滤后的pairs前五个: [['i m .', 'j ai ans .'], ['i m ok .', 'je vais bien .'], ['i m ok .', 'ca va .'], ['i m fat .', 'je suis gras .'], ['i m fat .', 'je suis gros .']]


In [12]:
# 对以上数据准备函数进行整合，并使用类Lang对语言对进行数据映射
def prepareData(lang1, lang2):
    """数据准备函数，完成将所有字符串数据向数值型数据的映射以及过滤语言对参数lang1, lang2分别代表源语言和目标语言的名字"""
    # 首先通过readLangs函数获得input_lang, output_lang对象，以及字符串类型的语言对列表
    input_lang, output_lang, pairs = readLangs(lang1, lang2)
    # 对字符串类型的语言对列表进行过滤操作
    pairs = filterPairs(pairs)
    # 对过滤后的语言对列表进行遍历
    for pair in pairs:
        # 并使用input_lang和output_lang的addSentence方法对其进行数值映射
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    # 返回数值映射后的对象，和过滤后语言对
    return  input_lang, output_lang, pairs

In [13]:
# 调用
input_lang, output_lang, pairs = prepareData('eng', 'fra')

In [14]:
print("input_n_words:", input_lang.n_words)
print("output_n_words:", output_lang.n_words)
print(random.choice(pairs))

input_n_words: 2803
output_n_words: 4345
['i m sorry you re leaving us .', 'c est triste que tu doives partir .']


In [15]:
# 将语言对转化为模型输入需要的张量
def tensorFromSentence(lang, sentence):
    """将文本句子转换为张量，参数lang代表传入的Lang的实例化对象，sentence是预转换的句子"""
    # 对句子进行分割并遍历每一个词汇，然后使用lang的word2index方法找到它对应的索引
    # 这样就得到了该句子对应的数值列表
    indexes = [lang.word2index[word] for word in sentence.split(' ')]
    # 然后加入句子结束标志
    indexes.append(EOS_token)
    # 将其使用torch.tensor封装成张量，并改变它的形状为nx1,以方便后续计算
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1,1)

def tensorFromPair(pair):
    """将语言对转换为张量对，参数pair为一个语言对"""
    # 调用tensorFromSentence分别将源语言和目标语言分别处理，获得对应的张量表示
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    # 最后返回它们组成的元组
    return (input_tensor, target_tensor)

# 输入参数，取pairs的第一条
pair = pairs[0]

# 调用
pair_tensor = tensorFromPair(pair)
print(pair_tensor)

(tensor([[2],
        [3],
        [4],
        [1]]), tensor([[2],
        [3],
        [4],
        [5],
        [1]]))


# 第三步：构建基于GRU的编码器和解码器

In [16]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        """它初始化参数有两个，input_size代表解码器的输入尺寸即源语言的词表大小，hidden_size
        代表GRU的隐藏层节点数，也代表词嵌入维度，同时又是GRU的输入尺寸"""
        super(EncoderRNN, self).__init__()
        # 将参数hidden_size传入类中
        self.hidden_size = hidden_size
        # 实例化nn中预定义的Embedding层，它的参数分别是input_size,hidden_size
        # 这里人词嵌入维度即hidden_size
        # nn.Embedding的演示在该代码下方
        self.embedding = nn.Embedding(input_size, hidden_size)
        # 然后实例化nn中预定义的GRU层，它的参数是hidden_size
        # nn.GRU的演示在该代码下方
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        """编码器前身逻辑函数中参数有两个，input代表源语言的Embedding层输入张量 hidden代表编码器层gru的初始隐层张量"""
        # 将输入张量进行embedding操作，并使其形状变为(1,1,-1),-1代表自动计算维度
        # 理论上，我们的编码器每次只以一个词作为输入，因此词汇映射后的尺寸应该是[1, embedding]
        # 而这里转换成三维的原因是因为torch中预定义gru必须使用三维张量作为输入，因此我们拓展了一个维度
        output = self.embedding(input).view(1, 1, -1)
        # 然后将embedding层的输出和传入的初始化hidden作为gru的输入传入其中
        # 获得最终gru的输出output和对应的隐层张量hidden,并返回结果
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        """初始化隐层张量函数"""
        # 将隐层张量初始化成1x1xself.hidden_size大小的0张量
        return  torch.zeros(1, 1, self.hidden_size, device=device)


# 实例化参数
hidden_size = 25
input_size = 20

# 输入参数
# pair_tensor[0]代表源语言即英文的句子，pair_tensor[0][0]代表句子中的第一个词
input = pair_tensor[0][0]
# 初始化第一个隐层张量，1x1xhidden_size的0张量
hidden = torch.zeros(1, 1, hidden_size)

# 调用
encoder = EncoderRNN(input_size, hidden_size)
encoder_output, hidden = encoder(input, hidden)
print(encoder_output)


tensor([[[ 0.0759,  0.2893, -0.4880, -0.1749, -0.3697, -0.2214, -0.3071,
          -0.2766,  0.3184, -0.1032,  0.1865,  0.2051, -0.2579, -0.0905,
          -0.1794,  0.2911,  0.1269,  0.0689,  0.2692, -0.0983,  0.1131,
          -0.0073, -0.2844, -0.2351,  0.2266]]], grad_fn=<StackBackward0>)


In [17]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        """初始化函数有两个参数，hidden_size代表解码器中的GRU输入尺寸，也是它的隐层节点数
        output_size代表整个解码器的输出尺寸，也是我们希望的指定尺寸即目标语言的词表大小"""
        super(DecoderRNN, self).__init__()
        # 将hidden_size传入到类中
        self.hidden_size = hidden_size
        # 实例化一个nn中的embedding层对象，它的参数output这里表示目标语言的词表大小
        # hidden_size表示目标语言的词嵌入维度
        self.embedding = nn.Embedding(output_size, hidden_size)
        # 实例化GRU对象，输入参数都是hidden_size,代表它的输入尺寸和隐层节点数相同
        self.gru = nn.GRU(hidden_size, hidden_size)
        # 实例化线性层，对GRU的输出做线性变化，获得我们希望的输出尺寸output_size
        # 因此它的两个参数分别是hidden_size,output_size
        self.out = nn.Linear(hidden_size, output_size)
        # 最后使用softmax进行处理，以便于分类
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        """解码器的前向逻辑函数中，参数有两个，input代表目标语言的Embedding层输入张量 hidden代表解码器GRU的初始隐层张量"""
        # 将输入张量进行embedding操作，并使其形状变为(1, 1, -1)，-1代表自动计算维度
        # 原因和解码器相同，因为torch预定义的GRU层只接受三维张量作为输入
        output = self.embedding(input).view(1, 1, -1)
        # 然后使用relu函数对输出进行处理，根据relu函数的特性，将使Embedding矩阵更稀疏，以防止
        output = F.relu(output)
        # 接下来，将把embedding的输出以及初始化的hidden张量传入到解码器gru中
        output, hidden = self.gru(output, hidden)
        # 因为GRU输出的output也是三维张量，第一维没有意义，因此可以通过output[0]来降维
        # 再传给线性层做变换，最后用softmax处理以便于分类
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        """初始化隐层张量函数"""
        # 将隐层张量初始化成为1x1xself.hidden_size大小的0张量
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [18]:
# 实例化参数
hidden_size = 25
output_size = 10

# 输入参数
# pair_tensor[1]代表目标语言即法文的句子，pair_tensor[1][0]代表句子中的第一个词
input = pair_tensor[1][0]
# 初始化第一个隐层张量 1x1xhidden_size的0张量
hidden = torch.zeros(1, 1, hidden_size)

# 调用
decoder = DecoderRNN(hidden_size, output_size)
output, hidden = decoder(input, hidden)
print(output)

tensor([[-2.5320, -2.4627, -2.1612, -2.4296, -2.4357, -2.1388, -2.0293, -2.1476,
         -2.3790, -2.4549]], grad_fn=<LogSoftmaxBackward0>)


<img src="/Users/zhangli/Library/Application Support/typora-user-images/image-20230703173240587.png">

In [22]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        """
        :param hidden_size: 代表解码器中GRU的输入尺寸，也是它的隐层节点数
        :param output_size: 代表整个解码器的输出尺寸，也是我们希望得到的指定尺寸即目标语言的词表大小
        :param dropout_p:代表我们使用dropout层时的置0率，默认为0.1
        :param max_length: 代表句子的最大长度
        """
        super(AttnDecoderRNN, self).__init__()
        # 将以下参数传入类中
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        # 实例化一个Embedding层，输入参数是self.output_size(目标语言的词汇总数)和self.hidden_size(和词嵌入的维度)
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        # 根据attention的QKV理论，attention的输入参数为三个Q、K、V
        # 第一步：使用Q和K进行attention权值计算得到权重矩阵，再与V做矩阵乘法，得到V的注意力表示结果
        # 这里常见的计算方式有三种
        # 1. 将Q、K进行纵轴拼接，做一次线性变化，再使用softmax处理获得结果最后与V做张量乘法
        # 2. 将Q、K进行纵轴拼接，做一次线性变化,再使用tanh函数，然后再进行内部求和，最后使用softmax处理获得结果再与V做张量乘法
        # 3. 将Q和K的转置做点积运算，然后除以一个缩放系数，再使用softmax处理获得结果最后与V做张量乘法

        # 说明：当注意力权重矩阵和V都是三维张量且第一维代表为batch条数时，则做bmm运算
        # 第二步，根据第一步采用的计算方法，如果是拼接方法，则需要将Q与第二步的计算结果再进行拼接
        # 如果是转置点积，一般是自注意力，Q与V相同，则不需要进行与Q的拼接，因此第二步的计算方式与第一步采用全值计算方法有关
        # 第三步，最后为了使整个attention结构按照指定尺寸输出，使用线性层作用在第二步的结果上做一个线性变换，得到最终对Q的注意力表示

        # 我们这里使用的是第一步中的第一种计算方式，因此需要一个线性变换的矩阵，实例化nn.Linear
        # 因为它的输入是Q、K拼接，所以输入的第一个参数是self.hidden_size * 2,第二个参数是self.max_length
        # 这里的Q是解码器的Embedding层的输出，K是解码器GRU的隐层输出，因为首次隐层还没有任何输出，会使用编码器的隐层输出
        # 而这里的V是解码器层的输出
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        # 接着我们实例化另外一个线性层，它是attention理论中的第四步的线性层，用于规范输出尺寸
        # 这里它的输入来自第三步的结果，因为第三步的结果是将Q与第二步的结果进行拼接，因此输入的维度是self.hidden_size * 2
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        # 接着实例化一个nn.Dropout层，并传入self.dropout_p
        self.dropout = nn.Dropout(self.dropout_p)
        # 之后实例化nn.GRU,它的输入和隐层的尺寸都是self.hidden_size
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        # 最后实例化gru后面的线性层，也就是我们的解码器输出层
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        """forward函数的输入参数有三个，分别是源数据输入张量，初始的隐层张量，以及解码器的输出张量"""
        # 根据结构计算图，输入张量进行Embedding层并扩展维度
        embedded = self.embedding(input).view(1, 1, -1)
        # 使用dropout进行随机丢弃，防止过拟合
        embedded = self.dropout(embedded)

        # 进行attention的权重计算，我们使用第一种方式
        # 将Q和K进行纵轴拼接，做一性线性变化，最后使用softmax处理获得结果
        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        # 然后进行第一步的后半部分，将得到的权重矩阵与V做矩阵乘法计算，当二者都是三维张量且第一维
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                  encoder_outputs.unsqueeze(0))
        # 之后进行第二步，通过取[0]是用来降维，根据第一步采用的计算方法，需要将Q与第一步的计算结果
        output = torch.cat((embedded[0], attn_applied[0]), 1)

        # 最后是第三步，使用线性层作用在第三步的结果上做一个线性变换并扩展维度，得到输出
        output = self.attn_combine(output).unsqueeze(0)

        # attention结构的结果使用relu激活
        output = F.relu(output)

        # 将激活后的结果作为gru的输入和hideen一起传入其中
        output. hidden = self.gru(output, hidden)

        # 最后将结果降维并使用softmax处理得到最终的结果
        output = F.log_softmax(self.out(output[0]), dim=1)
        # 返回解码器结果，最后隐层张量以及注意力权重张量
        return output, hidden, attn_weights

    def initHidden(self):
        """
        初始化隐层张量为1x1xself.hidden_size大小的张量
        :param self:
        :return:
        """
        return  torch.zeros(1, 1, self.hidden_size, device = device)

In [27]:
# 实例化参数
hidden_size = 25
output_size = 10

# 输入参数
input = pair_tensor[1][0]
print(input)
hidden = torch.zeros(1, 1, hidden_size)
print(hidden)
# encoder_outputs需要是encoder中每一个时间步的输出堆叠而成
# 它的形状应该是10x25,我们这里直接随机初始化一个张量
encoder_outputs  = torch.randn(10, 25)

# 调用
decoder = AttnDecoderRNN(hidden_size, output_size)
output, hidden, attn_weights = decoder(input, hidden, encoder_outputs)
print(output)
print(output.shape)
print(hidden.shape)
print(attn_weights)
print(attn_weights.shape)

tensor([2])
tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0.]]])
tensor([[-2.4642, -2.9059, -2.3662, -2.3890, -1.7398, -2.5366, -2.2330, -2.5282,
         -2.0142, -2.3074]], grad_fn=<LogSoftmaxBackward0>)
torch.Size([1, 10])
torch.Size([1, 1, 25])
tensor([[0.1039, 0.0769, 0.2070, 0.0836, 0.0378, 0.0322, 0.0350, 0.1376, 0.1054,
         0.1807]], grad_fn=<SoftmaxBackward0>)
torch.Size([1, 10])


## 第四步：构建模型训练函数，并进行训练
- 什么是teacher_forcing?
它是一种用于序列生成任务的训练技巧，在seq2seq架构中，根据循环神经网络理论，解码器每次应该使用上一步的结果作为输入的一部分，但是在训练的过程中，一旦上一步的结果是错误的，就会导致这种错误被累积，无法达到训练效果，因此，我们需要一种机制改变上一步出错的情况，因为训练时，我们是已知正确的输出应该是什么的，因此可以强制将上一步结果设置为正确的输出 ，这种方式就叫做teacher_forcing
- teacher_forcing的作用：
    - 能够在训练的时候矫正模型的预测，避免在序列生成的过程中误差进一步放大
    - teacher_forcing能够极大的加快模型的收敛速度，令模型训练过程更快更平稳

In [ ]:
# 设置teacher_forcing比率为0.5
teeacher_forcing_ratio = 0.5

def train(input_tensor, target_tensor, encoder, encoder_optimizer, decoder_optimizer)